# Install the prerequisites

In [ ]:
! pip install datasets
! pip install huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00


# Load data

In [ ]:
from datasets import load_dataset
dataset = load_dataset("AGBonnet/augmented-clinical-notes")

README.md:   0%|          | 0.00/7.76k [00:00<?, ?B/s]

augmented_notes_30K.jsonl:   0%|          | 0.00/372M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'] = dataset['train'].remove_columns(['note', 'conversation', 'summary'])

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'full_note'],
        num_rows: 30000
    })
})

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset['train'])

In [ ]:
df = df.drop_duplicates(subset=['full_note'])

In [ ]:
len(df)

24341

In [ ]:
df['text_length'] = df['full_note'].apply(len)

# DeCLUTR split

## The copied from repo necessary function


In [ ]:
from typing import Callable, List, Optional, Tuple

import numpy as np

def sample_anchor_positive_pairs(
    text: str
) -> Tuple[List[str], List[str], List[str]]:
    """Returns a `Tuple` of `List`s, containing `num_anchors` anchor spans and `num_positives`
    positive spans sampled from `text`.

    # Parameters

    text : `str`, required
        The string to extract anchor and positive spans from.
    num_anchors : `int`, required
        The number of spans to sample from `text` to serve as anchors.
    num_positives : `int`, required
        The number of spans to sample from `text` to serve as positives (per anchor).
    max_span_len : `int`, required
        The maximum length of spans, after tokenization, to sample.
    min_span_len : `int`, required
        The minimum length of spans, after tokenization, to sample.
    sampling_strategy : `str`, optional (default = `None`)
        One of `"subsuming"` or `"adjacent"`. If `"subsuming"`, positive spans are always subsumed
        by the anchor. If `"adjacent"`, positive spans are always adjacent to the anchor. If not
        provided, positives may be subsumed, adjacent to, or overlapping with the anchor.
    tokenizer : `Callable`, optional (default = `None`)
        Optional tokenizer to use before sampling spans. If `None`, `text.split()` is used.
    """
    # Tokenize the incoming text. Whitespace tokenization is much more straightforward
    # (we don't need to worry about chopping up subword tokens), but a user can also provide
    # their own tokenization scheme if they want.
    num_anchors = 2
    num_positives = 2 # maybe change
    max_span_len = 120 # maybe 256
    min_span_len = 10 # maybe change, 82 is average number of charact in a sentence
    sampling_strategy =  "subsuming" #"adjacent" None
    tokenizer = None


    tokens = text.split() if tokenizer is not None else text.split()
    tok_method = "tokenizer(text)" if tokenizer else "text.split()"
    num_tokens = len(tokens)

    if num_tokens < num_anchors * max_span_len * 2:
        raise ValueError(
            f"len({tok_method}) should be at least {num_anchors * max_span_len * 2}"
            f" (num_anchors * max_span_len * 2), got {num_tokens}."
        )
    if min_span_len > max_span_len:
        raise ValueError(
            f"min_span_len must be less than max_span_len ({max_span_len}), got {min_span_len}."
        )
    if max_span_len > num_tokens:
        raise ValueError(
            (
                f"max_span_len must be less than or equal to"
                f" len({tok_method}) ({num_tokens}), got {max_span_len}."
            )
        )

    # Valid anchor starts are token indices which begin a token span of at least max_span_len.
    anchors, positives = [], []
    anchor_positive_pairs = []
    valid_anchor_starts = list(range(0, num_tokens - max_span_len + 1, max_span_len))
    for i in range(num_anchors):
        # Sample the anchor length from a beta distribution skewed towards longer spans, the
        # intuition being that longer spans have the best chance of being representative of the
        # document they are sampled from.
        anchor_len = int(np.random.beta(4, 2) * (max_span_len - min_span_len) + min_span_len)
        # This check prevents an edge case were we run out of valid_anchor_starts.
        if len(valid_anchor_starts) // (num_anchors - i) < num_anchors - i:
            anchor_start_idx = np.random.choice([0, len(valid_anchor_starts) - 1])
        else:
            anchor_start_idx = np.random.randint(len(valid_anchor_starts))
        # When num_anchors = 1, this is equivalent to uniformly sampling that starting position.
        anchor_start = np.random.randint(
            valid_anchor_starts[anchor_start_idx],
            # randint is high-exclusive
            valid_anchor_starts[anchor_start_idx] + max_span_len - anchor_len + 1,
        )
        # Once sampled, remove an anchor (and its immediate neighbours) from consideration.
        del valid_anchor_starts[max(0, anchor_start_idx - 1) : anchor_start_idx + 2]
        anchor_end = anchor_start + anchor_len
        anchors.append(" ".join(tokens[anchor_start:anchor_end]))
        anchor = " ".join(tokens[anchor_start:anchor_end])
        # Sample positives from around the anchor. The intuition being that text that appears
        # close together is the same document is likely to be semantically similar.
        for _ in range(num_positives):
            # A user can specify a subsuming or adjacent only sampling strategy.
            if sampling_strategy == "subsuming":
                # To be strictly subsuming, we cannot allow the positive_len > anchor_len.
                positive_len = int(
                    np.random.beta(2, 4) * (anchor_len - min_span_len) + min_span_len
                )
                # randint is high-exclusive
                positive_start = np.random.randint(anchor_start, anchor_end - positive_len + 1)
            elif sampling_strategy == "adjacent":
                # Restrict positives to a length that will allow them to be adjacent to the anchor
                # without running off the edge of the document. If the anchor has sufficent room on
                # either side, this won't be a problem and max_positive_len will equal max_span_len.
                max_positive_len = min(max_span_len, max(anchor_start, num_tokens - anchor_end))
                if max_positive_len < max_span_len:
                    print(
                        (
                            "There is no room to sample an adjacent positive span. Temporarily"
                            " reducing the maximum span length of positives. This message will not"
                            " be displayed again."
                        )
                    )
                positive_len = int(
                    np.random.beta(2, 4) * (max_positive_len - min_span_len) + min_span_len
                )
                # There are two types of adjacent positives, those that border the beginning of the
                # anchor and those that border the end. The checks above guarantee at least one of
                # these is valid. Here we just choose from the valid positive starts at random.
                valid_starts = []
                if anchor_start - positive_len > 0:
                    valid_starts.append(anchor_start - positive_len)
                if anchor_end + positive_len <= num_tokens:
                    valid_starts.append(anchor_end)
                positive_start = np.random.choice(valid_starts)
            else:
                # Sample positive length from a beta distribution skewed towards shorter spans. The
                # idea is to promote diversity and minimize the amount of overlapping text.
                positive_len = int(
                    np.random.beta(2, 4) * (max_span_len - min_span_len) + min_span_len
                )
                # By default, spans may be adjacent or overlap with each other and the anchor.
                # Careful not to run off the edges of the document (this error may pass silently).
                positive_start = np.random.randint(
                    max(0, anchor_start - positive_len),
                    min(anchor_end, num_tokens - positive_len) + 1,  # randint is high-exclusive
                )

            positive_end = positive_start + positive_len
            positives.append(" ".join(tokens[positive_start:positive_end]))
            positive = " ".join(tokens[positive_start:positive_end])
            anchor_positive_pairs.append((anchor, positive))

    return anchors, positives, anchor_positive_pairs

## Apply

In [ ]:
df['num_tokens'] = df['full_note'].apply(lambda text: len(text.split()))

In [ ]:
df = df[df['num_tokens'] > 480]

In [ ]:
len(df)

11008

In [ ]:
df.head()

,idx,full_note,text_length,num_tokens
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281
1,77465,This is the case of a 56-year-old man that was...,3103,535
2,133948,A 36-year old female patient visited our hospi...,4378,655
3,80176,A 49-year-old male presented with a complaint ...,3866,636
4,72232,A 47-year-old male patient was referred to the...,3702,582


In [ ]:
#df.drop(columns=['anchors', 'positives'], inplace=True)

In [ ]:
df[['anchors', 'positives', 'anchor_positive']] = df['full_note'].apply(
    sample_anchor_positive_pairs
).apply(pd.Series)

In [ ]:
df.head()

,idx,full_note,text_length,num_tokens,anchors,positives,anchor_positive
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281,[the cause of her abnormal movements. Her MRI ...,"[copper levels, slit lamp microscopy for the K...",[(the cause of her abnormal movements. Her MRI...
1,77465,This is the case of a 56-year-old man that was...,3103,535,[double layers to correct the defect. The clin...,[correct the defect. The clinical course of th...,[(double layers to correct the defect. The cli...
2,133948,A 36-year old female patient visited our hospi...,4378,655,[range of motion (ROM) in the left hip joint p...,[An anteroposterior view of the left hip revea...,[(range of motion (ROM) in the left hip joint ...
3,80176,A 49-year-old male presented with a complaint ...,3866,636,[replacement system was used to perform the to...,[had full flexion and extension of the elbow a...,[(replacement system was used to perform the t...
4,72232,A 47-year-old male patient was referred to the...,3702,582,"[both knees over 1 year.\nIn September 2016, t...",[him from ambulation. The pain increased with ...,"[(both knees over 1 year.\nIn September 2016, ..."


In [ ]:
# Expand each list of anchor_positive pairs into separate rows
df_expanded = df.explode('anchor_positive')

# Then, split the pairs into 'anchor' and 'positive' columns
df_expanded[['anchor', 'positive']] = pd.DataFrame(df_expanded['anchor_positive'].tolist(), index=df_expanded.index)


In [ ]:
df_expanded["positive"].head(4).head(2).head(1)[0]

'copper levels, slit lamp microscopy for the KF ring, complete blood count, TLC, DLC, and USG'

In [ ]:
df_expanded["anchor"].head(4).head(2).tail(1)[0]

'the cause of her abnormal movements. Her MRI scan of the brain, serum and urine copper levels, slit lamp microscopy for the KF ring, complete blood count, TLC, DLC, and USG of the abdomen did not reveal'

In [ ]:
df_expanded.head()

,idx,full_note,text_length,num_tokens,anchors,positives,anchor_positive,anchor,positive
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281,[the cause of her abnormal movements. Her MRI ...,"[copper levels, slit lamp microscopy for the K...",(the cause of her abnormal movements. Her MRI ...,the cause of her abnormal movements. Her MRI s...,"copper levels, slit lamp microscopy for the KF..."
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281,[the cause of her abnormal movements. Her MRI ...,"[copper levels, slit lamp microscopy for the K...",(the cause of her abnormal movements. Her MRI ...,the cause of her abnormal movements. Her MRI s...,"blood count, TLC, DLC, and USG of the abdomen ..."
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281,[the cause of her abnormal movements. Her MRI ...,"[copper levels, slit lamp microscopy for the K...",(and abnormal contraction of the neck muscles ...,and abnormal contraction of the neck muscles t...,muscles that would pull her head to the right ...
0,155216,"A a sixteen year-old girl, presented to our Ou...",7979,1281,[the cause of her abnormal movements. Her MRI ...,"[copper levels, slit lamp microscopy for the K...",(and abnormal contraction of the neck muscles ...,and abnormal contraction of the neck muscles t...,her head to the right in an upward direction. ...
1,77465,This is the case of a 56-year-old man that was...,3103,535,[double layers to correct the defect. The clin...,[correct the defect. The clinical course of th...,(double layers to correct the defect. The clin...,double layers to correct the defect. The clini...,correct the defect. The clinical course of the...


In [ ]:
# Drop the intermediate column
df_expanded = df_expanded.drop(columns=['anchor_positive','anchors', 'positives'])

In [ ]:
df_expanded = df_expanded.drop(columns=['full_note'])

In [ ]:
df_expanded = df_expanded.drop(columns=['text_length',	'num_tokens'])
df_expanded['pairs_unique_id'] = range(0, len(df_expanded))

In [ ]:
df_expanded = df_expanded.reset_index(drop=True)

In [ ]:
df_expanded.head(10)

,idx,anchor,positive,pairs_unique_id
0,155216,the cause of her abnormal movements. Her MRI s...,"copper levels, slit lamp microscopy for the KF...",0
1,155216,the cause of her abnormal movements. Her MRI s...,"blood count, TLC, DLC, and USG of the abdomen ...",1
2,155216,and abnormal contraction of the neck muscles t...,muscles that would pull her head to the right ...,2
3,155216,and abnormal contraction of the neck muscles t...,her head to the right in an upward direction. ...,3
4,77465,double layers to correct the defect. The clini...,correct the defect. The clinical course of the...,4
5,77465,double layers to correct the defect. The clini...,correct the defect. The clinical course of the...,5
6,77465,9th rib and was lying even in two adjacent int...,one nodule in lower lobe of the right lung wit...,6
7,77465,9th rib and was lying even in two adjacent int...,rib and was lying even in two adjacent interco...,7
8,133948,range of motion (ROM) in the left hip joint pe...,An anteroposterior view of the left hip reveal...,8
9,133948,range of motion (ROM) in the left hip joint pe...,collapse of the femoral head and dysplasia of ...,9


In [ ]:
len(df_expanded)

44032

In [ ]:
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44032 entries, 0 to 44031
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   idx              44032 non-null  object
 1   anchor           44032 non-null  object
 2   positive         44032 non-null  object
 3   pairs_unique_id  44032 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


# Push to HUB

In [ ]:
!git config --global credential.helper store

In [ ]:
from datasets import Dataset, Features, Value
# Define the schema (optional but recommended)
features = Features({
    'pairs_unique_id': Value('int64'),
    'idx': Value('int64'),
    'anchor': Value('string'),
    'positive': Value('string')
})

# Convert the DataFrame to a Dataset
expanded_dataset = Dataset.from_pandas(df_expanded, features=features)

In [ ]:
expanded_dataset

Dataset({
    features: ['pairs_unique_id', 'idx', 'anchor', 'positive'],
    num_rows: 44032
})

In [ ]:
from datasets import DatasetDict
train_test_split = expanded_dataset.train_test_split(test_size=0.2, seed=42)

# Step 2: Further Split Train into Train and Validation (75% Train, 25% Validation from the 80% Train)
train_valid_split = train_test_split['train'].train_test_split(test_size=0.25, seed=42)

# Step 3: Combine splits into a DatasetDict
dataset = DatasetDict({
    'train': train_valid_split['train'],
    'val': train_valid_split['test'],  # This is the validation set
    'test': train_test_split['test']
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pairs_unique_id', 'idx', 'anchor', 'positive'],
        num_rows: 26418
    })
    val: Dataset({
        features: ['pairs_unique_id', 'idx', 'anchor', 'positive'],
        num_rows: 8807
    })
    test: Dataset({
        features: ['pairs_unique_id', 'idx', 'anchor', 'positive'],
        num_rows: 8807
    })
})

In [ ]:
dataset.push_to_hub("bebeyondo/medical-subsuming")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/bebeyondo/medical-subsuming/commit/64c5fc38ed0e945cf4631f2795dd504dc98d1680', commit_message='Upload dataset', commit_description='', oid='64c5fc38ed0e945cf4631f2795dd504dc98d1680', pr_url=None, pr_revision=None, pr_num=None)